In [ ]:
mari, juksemari = Contributor.objects.filter(display_name__icontains='Haanshus')

In [1]:
#finne personer som kan merges

from fuzzywuzzy import process
from django.db.models import Count
contributors = Contributor.objects.annotate(
        num_bylines=Count('byline')
    ).order_by('-num_bylines')

#one_hit_wonders = contributors.filter(num_bylines=1)
one_hit_wonders = []
with open('one_hit_wonders.txt') as fp:
    for line in fp.readlines():
        id = int(line.split()[0])
        one_hit_wonders.append(id)
        
    

In [2]:
names = sorted(contributors.values_list('display_name', 'num_bylines'

for n in names:
    print(n)
    


SyntaxError: invalid syntax (<ipython-input-2-28e44b990f82>, line 3)

In [3]:
nn = Contributor.objects.get(display_name="Nomen Nescio")
for bl in nn.byline_set.all():
    bylines = bl.story.byline_set.all()
    for byline in bylines:
        print(byline) 


@bl: Av: Nomen Nescio, KS)
@bl: Av: Nomen Nescio, LR)
@bl: Av: Nomen Nescio, HR)
@bl: Av: Nomen Nescio, NH)
@bl: Av: Nomen Nescio, PVRH)
@bl: Av: Nomen Nescio, IKL)
@bl: Av: Nomen Nescio, HR)
@bl: Av: Nomen Nescio, BHK)
@bl: Av: Nomen Nescio, Kjetil Strømme)
@bl: Av: Nomen Nescio, IKL)
@bl: Av: Nomen Nescio, BHK)
@bl: Av: Nomen Nescio, HR)
@bl: Av: Nomen Nescio, IKL)
@bl: Av: Nomen Nescio, HR)
@bl: Av: Nomen Nescio, BHK)
@bl: Tekst: Kjetil Strømme)
@bl: Foto: Berit Andreassen)
@bl: Av: Nomen Nescio, @maging: – Kan jeg skrive på russisk? undrer Nils Lid Hjort, professor i matematikk. Student Lars Sydnes sier ingen ting, han er allerede godt i gang med sin ode til kvinnen)
@bl: Av: Nomen Nescio, PVRH)
@bl: Av: Nomen Nescio, TL)
@bl: Av: Nomen Nescio, BHK)
@bl: Av: Nomen Nescio, HR)
@bl: Av: Nomen Nescio, Manus og regi: Bjarne Henning Kvåle. Kamera: Khai Nguyen)
@bl: Av: Nomen Nescio, PVRH)
@bl: Av: Nomen Nescio, HR)
@bl: Av: Nomen Nescio, PHP)
@bl: Av: Nomen Nescio, BHK)
@bl: Av: Nomen N

@bl: Av: Nomen Nescio, SEH)
@bl: Av: Nomen Nescio, AJ)
@bl: Av: Nomen Nescio, SEH)
@bl: Av: Nomen Nescio, AJ)
@bl: Av: Nomen Nescio, DSH)
@bl: Av: Nomen Nescio, KRN)
@bl: Av: Nomen Nescio, DSH)
@bl: Av: Nomen Nescio, KRN)
@bl: Av: Nomen Nescio, SB)
@bl: Av: Nomen Nescio, AJ)
@bl: Av: Nomen Nescio, SB)
@bl: Av: Nomen Nescio, AJ)
@bl: Av: Nomen Nescio, by: Naboens tre naboer (ved en av dem))
@bl: Av: Nomen Nescio, by: Leder i Studentparlamentet i Oslo Ingrid Stranger–Thorsen)
@bl: Av: Nomen Nescio, ES)
@bl: Av: Nomen Nescio, DSH)
@bl: Av: Nomen Nescio, KRN)
@bl: Av: Nomen Nescio, DSH)
@bl: Av: Nomen Nescio, KRN)
@bl: Av: Nomen Nescio, by: Lilly Susanne Stensland ( Foto: Balkaran Singh)
@bl: Av: Nomen Nescio, AJ)
@bl: Av: Nomen Nescio, IMJ)
@bl: Av: Nomen Nescio, IK)
@bl: Av: Nomen Nescio, IMJ)
@bl: Av: Nomen Nescio, IK)
@bl: Av: Nomen Nescio, by: Naboens tre naboer (ved en av dem))
@bl: Av: Nomen Nescio, by: Professor Emeritus Esther Bøk skriver for Universitas om vitenskap som forbigås 

@bl: Av: Nomen Nescio, CE)
@bl: Av: Nomen Nescio, by: Finn H. Eriksen (En av dem. Hemmelig))
@bl: Av: Nomen Nescio, text and styling: Camilla Svendsen Skriung Foto: Åshild Bekke Eidem Modeller: Dina Thomassen)
@bl: Av: Lance Morgas)
@bl: Av: Nomen Nescio, AGI)
@bl: Av: Nomen Nescio, HH)
@bl: Av: Nomen Nescio, AGI)
@bl: Av: Nomen Nescio, HH)
@bl: Av: Nomen Nescio, AJL)
@bl: Av: Nomen Nescio, ÅBE)
@bl: Av: Nomen Nescio, AJL)
@bl: Av: Nomen Nescio, ÅBE)
@bl: Av: Nomen Nescio, AJL)
@bl: Av: Nomen Nescio, ÅBE)
@bl: Av: Nomen Nescio, AJL)
@bl: Av: Nomen Nescio, ÅBE)
@bl: Av: Nomen Nescio, AGI)
@bl: Av: Nomen Nescio, HH)
@bl: Av: Nomen Nescio, MNP)
@bl: Av: Nomen Nescio, CE)
@bl: Av: Nomen Nescio, AGI)
@bl: Av: Nomen Nescio, HH)
@bl: Av: Nomen Nescio, MNP)
@bl: Av: Nomen Nescio, CE)
@bl: Av: Nomen Nescio, AGI)
@bl: Av: Nomen Nescio, HH)
@bl: Av: Nomen Nescio, MNP)
@bl: Av: Nomen Nescio, CE)
@bl: Av: Nomen Nescio, AGI)
@bl: Av: Nomen Nescio, HH)
@bl: Av: Nomen Nescio, MNP)
@bl: Av: Nomen Nesci

@bl: Av: Lektorprogrammets Programutvalg, ved leiar Ane Krogsæter Aarre)
@bl: Av: Nomen Nescio, by: Henrik Asheim er leder i Unge Høyre og gjesteskribent i Universitas)
@bl: Av: Sine Astad, leder av studentforeningen ved Handelshøyskolen BI i Oslo)
@bl: Av: Nomen Nescio, SBIO)
@bl: Tekst: Halvor Kongshavn, overbibliotekar ved Humsam-biblioteket)
@bl: Av: Nomen Nescio, by: Bibliotek for humaniora og samfunnsvitenskap: Halvor Kongshavn, overbibliotekar ved Humsam-biblioteket)
@bl: Av: Nomen Nescio, UB)
@bl: Tekst: Halvor Kongshavn, overbibliotekar ved Humsam-biblioteket)
@bl: Av: Nomen Nescio, by: Bibliotek for humaniora og samfunnsvitenskap: Halvor Kongshavn, overbibliotekar ved Humsam-biblioteket)
@bl: Av: Nomen Nescio, UB)
@bl: Av: Nomen Nescio, by: Iver Aastebøl er leder i Rød Ungdom og gjesteskribent i Universitas)
@bl: Av: Nomen Nescio, by: Velferdstingets Arbeidsutvalg ved leder Mari Berdal Djupvik)
@bl: Av: Nomen Nescio, by: Aksel Braanen Sterri er leder i Studentparlamentet ved 

In [ ]:
for id in one_hit_wonders[:10]:
    person = Contributor.objects.get(pk=id)
    story = person.byline_set.first().story
    #print(story)
    print(story.id)
    print(person)
    print(json.loads(story.legacy_html_source)[0]['fields']['byline'])
    print(story.legacy_prodsys_source)

In [ ]:
import json
list(story.bylines.all())

In [ ]:
everyone = Contributor.objects.values_list('display_name', flat=True)
for contributor in contributors[:100]:
    best = process.extract(contributor.display_name, everyone, limit=2)
    print(best)
    

In [1]:
from collections import Counter
def duplicates(queryset, attr, order_by=()):
    counter = Counter(queryset.values_list(attr, flat=True))
    duped_values = [el for el in counter if counter[el] > 1]
    sets = []
    for value in duped_values:
        items = queryset.filter(**{attr: value}).order_by(*order_by)
        sets.append(items)
    return sets

duplicates(ImageFile.objects.all(), '_md5')

[<ImageFileQuerySet [<ImageFile: 26-valgfusk2-AS.jpg>, <ImageFile: 14petit.jpg>, <ImageFile: 20-anm-uri.jpg>, <ImageFile: 12-jurk-01-AJ.jpg>, <ImageFile: 08-illustrasjon.jpg>, <ImageFile: 11-bolig.jpg>, <ImageFile: 34-Mag-mote-04.jpg>, <ImageFile: 11-eika-01b-KK.jpg>, <ImageFile: 17-Mote-01-MH.jpg>, <ImageFile: 22-bakside01-GKJ.jpg>, <ImageFile: 24anmKleen.jpg>, <ImageFile: 08-Blackstrap.jpg>, <ImageFile: 17-it-01-RE.jpg>, <ImageFile: 25mag-petit.jpg>, <ImageFile: 25-trefavo.jpg>, <ImageFile: 28-nyh-aasland.jpg>, <ImageFile: 03-kult-amatoren7-KB.jpg>, <ImageFile: 08-kenya-rrj.jpg>, <ImageFile: 09-toreaasland01-KB.jpg>, <ImageFile: 26-nmh-SL.jpg>, '...(remaining elements truncated)...']>,
 <ImageFileQuerySet [<ImageFile: 0roe.jpg>, <ImageFile: roe.jpg>]>,
 <ImageFileQuerySet [<ImageFile: BRANN3.jpg>, <ImageFile: 26-NYH-nyharbeidsmo-03-HB.jpg>, <ImageFile: 29-KUL-kulenfinkaff-09-HB.jpg>, <ImageFile: 29-KUL-kulenfinkaff-14-HB.jpg>, <ImageFile: 32-KUL-kul32foreles-01-EP.jpg>, <ImageFile: 3

In [ ]:
dupe_files = _
for qs in dupe_files:
    print()
    for img in qs:
        print(img, img.md5)
        #print('{id} {_md5} {_size:>8} {full_width:>4}x{full_height:<4} {imghash} {source_file}'.format(**img.__dict__))
    


26-valgfusk2-AS.jpg 5785393378aa42d6dc92896f9054288b
14petit.jpg b9236b5877039ec5561ea621e843990b
20-anm-uri.jpg 33381b1b08c6c06ee489c2c7e021cfe0
12-jurk-01-AJ.jpg 2e189f92d8ac397b5ffd75259b37eba5
08-illustrasjon.jpg 83a509da0ae2dc52387634f032c7489c
11-bolig.jpg b76accaa0f2fb8a8c9c83f0c9a73bae5
34-Mag-mote-04.jpg 2e06c65e8afa91f11eeec601730704b6
11-eika-01b-KK.jpg a2735415a1d089323170207214678d7f
17-Mote-01-MH.jpg 7c706b339ed6918e37de8c71ed36014c
22-bakside01-GKJ.jpg 121be301f672a6c577814c9d5ebf3613
24anmKleen.jpg 1dd16a45e71b8fd85830073595be3840
08-Blackstrap.jpg b79e531f5d0081819434bf3b79dae67d
17-it-01-RE.jpg edd3ccabb8c249e7b7a344a8cefa9a8a
25mag-petit.jpg 2ac2336517e0d45aeefec2fe3317ba10
25-trefavo.jpg 246cb35f1b42ae7cb4e41c664b8c6cdb
28-nyh-aasland.jpg 24acc67de0c2e96cf0767aa3c71d123e
03-kult-amatoren7-KB.jpg 186a900c29087e93d5a6d406ec98bb77
08-kenya-rrj.jpg c73fa695596da03b757bcd1c23e606da
09-toreaasland01-KB.jpg eb34a6b39dc810bccd170a2c7b5ec82d
26-nmh-SL.jpg cae4bb3e12cad31623

22-studenttv3-B-AS.JPG aed79822450fbe8a871aa71100085bdd
23-Lanekassen2-AS.JPG d2fd908b4a26631855aa941a8483c181
23-ansa1-MT.jpg ccf6a63201f22bb6063d77b223881d91
23-forskning.jpg 8c95705ade9608d2f4304dc5ba16f063
23-kopi-AS.jpg 64406e6c0ad685c9318eb76346e7afe4
23-rektorduell.JPG 02c9b0755dd56754178c5d7c2d3e2a79
23-studentparlamentet-AS.JPG b813023766acdd9e706f5d5289114d79
kvinnetatt-2.JPG 9628cb4f2656f98945a61af6af29c707
20-wildenvey5-AS.JPG a2447c1121b4f99f3a574c05c42d8769
22-litteraturhist-AS.JPG 277bb62ec02b4fbe5588b01b970c7ced
22-musikk1-ST.jpg 934986134ae18157b570d051fb62fcd7
drodle-prof-FA.jpg c8ec0668a4360e756297cf6442e80801
drodle-04-FA.jpg 0cdc8f909b5925e15a55cabe79fb200d
drodle-02-FA.jpg d6b327a197f28fd1b614d90fe04176f7
drodle-01-FA.jpg feaae97f823a0f0595ab7742e35e7ebd
drodle-03-FA.jpg d84f8b0da4acb7485b28f2cd5fb2eb56
wolf-01-FA.jpg af354ff41d3946a0cbed0663a8808465
baksiden2-AS.JPG 7f558ff0775d313951a590250375e79e
forsker-3-AH.JPG 02fcc09335d557370720171d940652cd
16anb-lionsSH.J

29-fredsfestival1-AS.JPG ebebfcdc8b7b715c81a5c95d2c4e4276
29-usitfyr-AS.JPG b1fc3af43c70e896d8e5c3b80a2c93db
30-afrika-AH.JPG 788e02fa0917eea8804faa90663153b3
30-palestinapol-AS.JPG 986f806c0da34edfe0abe1450bf99c98
30-clemet-1-AH.JPG d08ef4f9be1d6aff5629ab0e5076e30d
30-data-AS.JPG 4ff075cb8e8d64abb60586f93aa15fb6
30-hovedfag3-AS.JPG 928cbd6ea91848bf080fc6de07ab710e
30-hovedfag1-HM.JPG 3adb1d91e5b246ae0064dae4b3a484a7
30-stamcelle-2-B-AH.JPG 31ea93ba3a20aa98374ab8fb4b2a1efa
30-studentdotno5-HM.JPG 785e2baba42d6647885677eaae29e5b0
30-studentdotno1-HM.JPG 20d0ff542c9cc286e5adeaa2b26791fb
31-veteriner-4-AH.JPG c33bfdc4b3870f02072230f4775c4912
31-bondevik2-mf-st.JPG ebd5b53054965a4182f447e6dfa5c8b7
31-studentno-AS.JPG 41ec8fee6fa358b4b71f937787812082
31-tverrfaglig-AS.JPG f0072fbcfb9efa82f8503db27dfb2ace
32-luftslott-st.JPG 71158ba3790c0bcd24fe21ddeb9e3787
32-sykstud1.JPG 9bd579fe0bd330d576f6f5c201e9d045
32-johansen-torill-st.JPG 1ff8e6cc4c649b9e3aeaaa52faa5dcc5
32-dinamo-HM.JPG 3003d0c22aa

11kommentar.jpg d3821ffe047a6840fefafb0c3d28584f
11Clemet1ED.jpg 68dcff6e642b2628519c5578b4530b0e
11operaballED.jpg e89af91ad5166a3637e164c897cc37c4
12comment.jpg 1df6c2331059ebb165a9140b89ee7d3a
12selvangivelse1ED.jpg b121605c8550f5c9f12efeddc88fcc55
12karrierefyren1ED.jpg bcc1d62e473964c03ffa838aae65fba8
12magblaa.jpg 7c6959d1df442c4010478f5cc3b4fb66
magasinetvegardhole.jpg dbe7bfb66c5bd68b24d626026f6efba8
08-lege.jpg d97ccc669ae599eeacbb3fb8f1fd5543
08-ander-aukland-AS.jpg 8daa771767385e295f36cacf4b74e984
08-jussdoku-3-AH.jpg b22c05a853799fd0647ff1d51d5697c0
08-prosedyre3-AS.jpg 412b7fc29aa342a674bfaf6e57a79c94
09-skitur5-AS.jpg 8d97f8af035a6f6b6f9cd2534ecc2550
11-kunf-fu-AH.jpg 8efc096a7a2a75af4cda3ee4ba270e14
12-zoo-pri-05-2sp.jpg 125ad55a8c20f5f182e1d56928c37a10
12-zoo-pri-02-HM-2sp.jpg f1431bfaff909586a7bde49816e8431f
08-kj-bfnnskvot-EJ.jpg b00621a8d3176a2b8458a3ea6e856c0d
08-JST-1-AH.jpg 662a9050ab7eb74ffa6f11e53de7f45b
09-nsu-01-HM.jpg bd8fc0de2fbaedd380bfbeeed41c27a2
09-nazif

16p-8clesesalen1ED.jpg 71b4829240ed3486e8e65eae3c610108
16p-8clesesalen2ED.jpg 2d5f7e86a03dacac3674744a2a8de31e
l-8coppe.jpg 7e591e8a33d361a0bfaa1201a868904d
16-karlsenanmeld.jpg a700c860f692f0ad60b49b6b86f0a374
16-spananmeld.jpg b854e3912e49fb3967806f2eb65523cf
16-tjonnanmeld.jpg dd4c84ac80d3f2eca79397070cc064ed
18-anm-kroppenmin.jpg db8a8b9b5643d77555d30a918d4508b6
18-anm-cadillac.jpg 71626adf31c9e95c82b045bfb521f09e
18-anm-godo.jpg 379ca74afb1c7b34a85fcef7e5d5b1b6
18-Doktor-Kosmos01-KRN.jpg b57e6f3e86154519b7f62940884398d4
18-anm-lagaan.jpg 42dcbd9117cb661b36113f780eb493ab
18-anm-fossum.jpg 705fe72ee9572cd3e60bf82cf1d8a622
studteater2-EJ.jpg 5c87f30d3dc5edc1878299435fd88190
19jusstudine2ED.jpg 41d6326da3f45ebcd3dcad96dc11ad32
18forsikring2ED.jpg 3ec03db231894b2d9b3c5453f5db9836
19bjolsen3ED.jpg c2f823d21d4c61a51cd14666531a9278
19anmHonore.jpg 5876e02e25944558efb6b010b0e97a82
MotorpsychoEP.jpg e01da13974c144f5135172a277abd1ac
20kommentar2.jpg 9f2e6b76bc43979e63a87c2f2c14d4a9
21ADsioh

...

33-kult-tidsskrift8.jpg 4753afa8ec31b0ef7575eaac174305c0
33-kult-tidsskrift9.jpg d698e253ff469665be5bdd52c6bc0064
33-minstudietid-danielrybakken2.jpg 60963b10eb23aa207896d8ba7fc9a9e6
33-nyh-biverv2-Web-SL.jpg b0e88cb726a0f490510d184c4eab2caf
33-nyh-biverv3-Web-SL.jpg ed4ecaab359c194556d82434b74b011e
33-nyh-biverv5-Web-SL.jpg 68da16aef9f05bdced2e7f5590862d71
33-nyh-biverv-Web-SL.jpg 8b9f2012f24e28c68edac854ff4b0870
33-nyh-farmasi6-web-SR.jpg ebd29cb5e9a01e6c44ac157ad8122f32
33-nyh-papirsak1-web-SR.jpg 5fc02a406876aac3a0f55cf2c87a5c3d
33-nyh-papirsak2-web-SR.jpg 9694804eea06600fcd8f2d2366d3767b
33-nyh-papirsak3-web-SR.jpg e9d23211a01efe7f5b80711ddedb48d3
33-nyh-papirsak4-web-SR.jpg 9ad19618ee33c67cd641845bbf81ff9f
33-nyh-regjeringen1-Web-SL.jpg 3230ddd599f5b5f03aa379bb39709cc3
33-nyh-trening-Web-ArkivRE.jpg 936054e1667d7d3ade6bfbd21c075a56
33-petit.jpg da30a03af2a10b18694e97769ff305cd
33-stud-silje-cinnamon-Web-LRBL.jpg 2be7c9cb4e3c73181186eaddc244962e
33-stud-silje-eggsalg-Web-LRBL.jpg 

35-nyh-faks-nr06-2007.jpg af8151757587514501aa6674a0b0a3ef
35-nyh-faks-nr08-2008.jpg 672df0a05c74f09261a605fe3e6bceb8
35-nyh-faks-nr12-2008.jpg d8ebd53b5a39b876a965cd6b1be537d6
35-nyh-faks-nr13-2002.jpg aae712470daf181672de20402c94a1c6
35-nyh-faks-nr14-2000.jpg ebb3dc7d88186420eda1114e5305a4d8
35-nyh-faks-nr14-2001.jpg 11cda7ef5d455d3b74aa310daa0f136f
35-nyh-faks-nr15-2001.jpg 09821514a154051b26217a3cab8a8a60
35-nyh-faks-nr17-2007.jpg 834f6820bff4982acb14eada5874ff75
35-nyh-faks-nr18-2006.jpg ae682738817005591f2c266c6446ac5c
35-nyh-faks-nr19-2000.jpg cfc848c36c7eae1dcf1578893a7c2513
35-nyh-faks-nr19-2002.jpg 54e7e0e0c8650b46e6129b3196c8638b
35-nyh-faks-nr20-2001.jpg 492372dd11c1d879cf55ac5a722d1a7b
35-nyh-faks-nr20-2008.jpg deaba23fc65c31cc7cad523751316f59
35-nyh-faks-nr24-2002.jpg 35baca37e6a05111059a431f2a5c2c92
35-nyh-faks-nr24-2004.jpg 2594e0ebb85708368245298d68cd9b9f
35-nyh-faks-nr24-2005.jpg b21972d1c00e81267fa3f0e5a9546212
35-nyh-faks-nr24-2007.jpg 145f41af0947b108d2e4da9456f21c

03-anm-pony-the-pirate-02-Web-BO.jpg 208f059937480f4786c8dabd09670892
03-anm-pony-the-pirate-03-Web-BO.jpg 8e8d564aa72f6e48b99dd4a22b7f8a62
03-anm-pony-the-pirate-04-Web-BO.jpg 60a0a339f24c1f0b8e09bd70cfd7c6e0
03-anm-pony-the-pirate-05-Web-BO.jpg ecdd1802e7d5b46eaec724ffdad196dd
03-anm-pony-the-pirate-06-Web-BO.jpg 5e36c80b1f3b5cf66a276e1cb372c9b1
03-anm-pony-the-pirate-07-Web-BO.jpg 8de3fd6f54b389c693ddc42941384b0e
03-anm-14dikt-web.jpg 7c984061eac462b80fa60420e00b199c
03-anm-risk.jpg 3d658926396007dd9edc28f138754213
min-studietid-web-DSC2219.jpg 77b4d8306a698216f052fdd57cea4389
nyhetskommentarkonte.jpg 71834b35b5dfad68c52611578080dd71
akademika.jpg 18598220e8352189417f8f481e420d7a
justinEdwardk662.jpg eaf0772a6b0f0b3f19f3a62e24927d1a
hathawaySchmidtWarren.jpg bcebc5bb93cbd94327cd07ab44e0145f
koben.jpg 5ac0a04e2c4bff0f3eb372b6def65052
kobenhavn.jpg 9a6889f3dd947f972c77d462b216e15b
viselgerud.jpg 6005a9b26f892d80e8ef1fa21b3237cd
01-kul-nmh-eirik-birkeland-web.jpg 75c82977c877e28552fef7

04-nyh-spesped02-web-KB.jpg a388a0ad83859fed867bc3e458b05c3a
04-nyh-gaverogdonasjoner01-KB.jpg 7161c525b5acfde40ffec782a108f0cf
01-varoy-web-KB.jpg b94b5c96796fb16126b521c2261c9c3e
15-nyh-veterinarh-4-web-AAAH.jpg ae69f897e8dcd4b2feffb5cd72dfe5e8
Henry-David-Thoreau.jpg c3e22dd19617a847d9b90c96eef6c9fa
05-anm-Vox-humana.jpg 27c049995de50b06beaac8aa6a2e819c
05-anm-beneva.jpg b87e8365c8c7baa58c2a01271fd56c15
05-anm-berlinogvox.jpg e15910f21f36c03ece7815913b0a2ed3
05-anm-forflytninger.jpg 18aeabcacd6272ebcc58792eaf0b36d4
05-anm-muhlesen.jpg 88a497b1d2c9a97c884f70423ca03c32
05-anm-stiegler.jpg e8db539b90d2da014e934b0791974673
05-anm-thewolfman.jpg 651e838fa4c0ef974e70717d4587ffda
05-khio-teater01-web-KB.jpg 9aff74fa761eb2314465fc60aff991b5
05-khio-teater02-web-KB.jpg 642f58a2e0eb833cb14f34c6f1069810
05-khio-teater03-web-KB.jpg 4e2297153a199b0f7ade9de4db0b5256
05-khio-teater04-web-KB.jpg ea85bde85c9afd994c630b3937e2b9a5
05-khio-teater05-web-KB.jpg 3f28309772ac73733bc9cf522ed17bad
05-khio-te

nyhetskommentarnedkvitne.jpg e27df7d6c72dfdd215064e229bc21f3b
07nyhetskommentar.jpg 9a0863c4c533b962faa7b045db8249d9
sirkusnedkvitne.jpg e27df7d6c72dfdd215064e229bc21f3b
07-anm-thelittlehandsofasphalt02-AN.jpg 0392880082a70df920b7ed591b7b003a
07-anm-thelittlehandsofasphalt03-AN.jpg 66b63060e992d0c056ab816532a5f5bc
07-anm-ungdomskulen01-AN.jpg 32948c4f47d68a5a3d67fe31eecdb55f
07-anm-ungdomskulen02-AN.jpg 7238ea53b326911d526ff6a1c5f85351
07-anm-ungdomskulen03-AN.jpg d37cabce35398ff2eec4710eea2cb34e
07-anm-ungdomskulen04-AN.jpg 091c1ba5fb45e30dcdcdbf1ebff11b04
07-anm-valkyrienallstars01-AN.jpg 11a1d09ab0bf1f0cc26d16f0573dfac8
07-anm-valkyrienallstars03-AN.jpg 58dfc492487a4e94d9a3aad85f4c83cb
07-anm-valkyrienallstars04-AN.jpg 2aa8830905f84a9b51daa82aabefda2b
07-anm-valkyrienallstars06-AN.jpg 2b8e1a192a3282fc27ef28a1c2d2f0c4
07-anm-valkyrienallstars07-AN.jpg 5ede205d635970d4ee0c064d49c78790
07-kultur-EiaOgIhle01-HL.jpg c418a415a5caef59fa6bb8137cbe0a0a
capetown4.JPG 42affaff4e97569c65d72d302

In [ ]:
for img in dupe_files[0]:
    img.save()

In [ ]:
from django.db import transaction
from django.apps import apps
from django.contrib.contenttypes.fields import GenericForeignKey
from django.db.models.fields.related import ManyToManyField

@transaction.atomic()
def merge_instances(primary_object, *alias_objects):
    """Merge several model instances into one, the `primary_object`.
    Use this function to merge model objects and migrate all of the related
    fields from the alias objects the primary object.
    Usage:
        from django.contrib.auth.models import User
        primary_user = User.objects.get(email='good@example.com')
        duplicate_user = User.objects.get(email='good+duplicate@example.com')
        merge(primary_user, duplicate_user)
    Based on: https://djangosnippets.org/snippets/382/
    """
    generic_fields = get_generic_fields()

    # get related fields
    many_to_many_fields, related_fields = discrimine(
        lambda field: isinstance(field, ManyToManyField),
        primary_object._meta._get_fields(forward=False, include_hidden=True)
    )

    # Loop through all alias objects and migrate their references to the
    # primary object
    for alias_object in alias_objects:
        # Migrate all foreign key references from alias object to primary
        # object.
        for related_object in related_fields:
            # The variable name on the alias_object model.
            alias_varname = related_object.get_accessor_name()
            # The variable name on the related model.
            obj_varname = related_object.field.name
            related_objects = getattr(alias_object, alias_varname)
            for obj in related_objects.all():
                setattr(obj, obj_varname, primary_object)
                obj.save()

        # Migrate all many to many references from alias object to primary
        # object.
        for related_many_object in many_to_many_fields:
            alias_varname = related_many_object.get_accessor_name()
            obj_varname = related_many_object.field.name
            related_many_objects = getattr(alias_object, alias_varname)
            for obj in related_many_objects.all():
                getattr(obj, obj_varname).remove(alias_object)
                getattr(obj, obj_varname).add(primary_object)

        # Migrate all generic foreign key references from alias object to
        # primary object.
        for field in generic_fields:
            filter_kwargs = {}
            filter_kwargs[field.fk_field] = alias_object._get_pk_val()
            filter_kwargs[field.ct_field] = field.get_content_type(alias_object)
            related_objects = field.model.objects.filter(**filter_kwargs)
            for generic_related_object in related_objects:
                setattr(generic_related_object, field.name, primary_object)
                generic_related_object.save()

        if alias_object.id:
            alias_object.delete()

    return primary_object


def get_generic_fields():
    """Return a list of all GenericForeignKeys in all models."""
    generic_fields = []
    for model in apps.get_models():
        for field_name, field in model.__dict__.items():
            if isinstance(field, GenericForeignKey):
                generic_fields.append(field)
    return generic_fields


def discrimine(pred, sequence):
    """Split a collection in two collections using a predicate.
    >>> discrimine(lambda x: x < 5, [3, 4, 5, 6, 7, 8])
    ... ([3, 4], [5, 6, 7, 8])
    """
    positive, negative = [], []
    for item in sequence:
        if pred(item):
            positive.append(item)
        else:
            negative.append(item)
    return positive, negative
